In [1]:
!pip install -q accelerate transformers bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.6 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import bitsandbytes as bnb

In [6]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("rombodawg/Llama-3-8B-Instruct-Coder")
# model = AutoModelForCausalLM.from_pretrained("rombodawg/Llama-3-8B-Instruct-Coder")

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
# using bitsandbytes cconfiguration to load model in 4bit
# bitsandbytes parameters
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_use_double_quant=use_nested_quant,
)
model = AutoModelForCausalLM.from_pretrained(
    "rombodawg/Llama-3-8B-Instruct-Coder",
    quantization_config=bnb_config,
    device_map="auto"
)

config.json:   0%|          | 0.00/739 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00004.bin:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00004.bin:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

pytorch_model-00003-of-00004.bin:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

pytorch_model-00004-of-00004.bin:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/143 [00:00<?, ?B/s]

In [ ]:
# # this way we can also download the retrained model but`load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions.
# model = AutoModelForCausalLM.from_pretrained("TheBloke/CodeLlama-13B-Instruct-fp16",
#                                              load_in_4bit = True,
#                                              device_map="auto",
#                                              torch_dtype ="auto"
#                                              )

In [8]:
prompt = "Write a python RestAPI function for calling a weather API"

In [9]:
inputs = tokenizer(prompt,return_tensors="pt").to(model.device)

In [10]:
inputs

{'input_ids': tensor([[128000,   8144,    264,  10344,   9240,   7227,    734,    369,   8260,
            264,   9282,   5446]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [11]:
prompt_len = inputs["input_ids"].shape[-1]

In [12]:
outputs = model.generate(**inputs, max_new_tokens = 150)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


In [13]:
code =tokenizer.decode(outputs[0][prompt_len:])

In [14]:
print(code)



Here is a simple example of a REST API function in Python using Flask that calls the OpenWeatherMap API to retrieve the current weather for a given city:

```Python
from flask import Flask, jsonify, request
import requests

app = Flask(__name__)

@app.route('/weather', methods=['GET'])
def get_weather():
    city = request.args.get('city')
    api_key = 'YOUR_OPENWEATHERMAP_API_KEY'
    url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        weather_data = response.json()
        return jsonify({'weather': weather_data['weather'][0]['description
